In [23]:
import pandas as pd
import numpy as np
import gamspy
import math

In [24]:
def binomial (n, p, size):
    return np.random.binomial(n, p, size)

def binomial_pmf(n, p, k):
    # Calculate the binomial coefficient
    binomial_coefficient = math.comb(n, k)
    # Calculate the probability of k successes
    probability_k_successes = binomial_coefficient * (p ** k) * ((1 - p) ** (n - k))
    return probability_k_successes

def randint (low, high, size):
    return np.random.randint (low, high, size)

# QUESTION 1
There are n products to be produced $\newline$
There are m suppliers which provide the parts to produce the products $\newline$
A product i needs $a_{ij}$ parts ($a_{ij >= 0}$, $i = 1\rightarrow n$, $j=1\rightarrow m $) $\newline$
The preorder cost for part $j$ is $b_j$ $\newline$ 
The demand for the products is random vector $D = (D_1,..., D_n)$ $\newline$
Cost additionaly $l_i$ to satisfy a unit of demand for product $i$, and unit selling price of that unit is $q_i$ $\newline$
The number of parts ordered is $x_j$, the number of units produced is $z_i$, the numbers of remaining parts unused is $y_j$


### Second stage
The second stage is mentioned first to emphasize the importance of understanding the uncertainty in case of solving a stochastic problem. This approach aligns with the principles of stochastic programming, where the second-stage decisions are made after observing the realization of random data and are contingent on the first-stage decisions. $\newline$
In this stage, we want to minimize the cost of production. Specifically, we need to find the optimal cost of filling an additional portion of demand while reselling the salvage parts to minimize the loss of not using the parts. $\newline$
$$ 
min_{x,y} \text { } Z=\sum^n_{i=1} (l_i-q_i)z_i - \sum^m_{j=1}s_jy_j \newline \text {subject to} \newline
y_j=x_j- \sum_{i=1}^n a_{ij}z_i, j=1,...,m \newline
0 \leq z_i \leq d_i, i=1,...,n \newline
y_j \geq 0, j=1,...,m
$$

### Big problem
We need to solve this
$$
min \text { } b^Tx + \sum^S_{i=1} p_s[(l_i-q_i)^Tz_i - s^T_iy_i] \newline
\text {subject to} \newline
y \geq 0 \newline
0 \leq z \leq d \newline
x \geq 0 \newline
$$

### Initiating parameters

In [25]:
n   = 8           # number of products
m   = 5           # number of parts to be ordered
S   = 2           # number of scenarios
p_s = [0.5, 0.5]  # probability of each scenario / density

D = np.array([binomial (10, 0.5, n),
              binomial (10, 0.5, n)]) # demand for each product in each scenario matrix [n x S]
p_d = []
for s in range(S):
    s_array = []
    for i in range(n):
        s_array.append(binomial_pmf(10, 0.5, D[s][i]))
    p_d.append(s_array)
p_d = np.array(p_d)

b = randint(50, 100, m) # preorder cost for each part

A = []
for i in range(n):
    A.append(np.array(randint(1,10,m)))

A = np.array(A) # bill of materials matrix [m x n]

s = randint (100, 200, m)       # salvage values
l = randint (150, 250, n)    # additional costs
q = randint (200, 400, n)  # unit selling prices

c = l-q #cost coefficients

In [26]:
print('Preorder cost: ', b)
print('Material usage:\n', A)
print('Salvage value: ', s)
print('Additional producing cost: ', l)
print('Selling prices: ', q)
print('Cost coefficient:', c)
print('Random demand:\n', D)
print('PMF of demand:\n', p_d)
# print('Demand:\n', D)

Preorder cost:  [59 69 52 62 87]
Material usage:
 [[9 4 7 8 7]
 [7 9 2 5 3]
 [8 2 9 7 3]
 [1 6 1 6 1]
 [3 3 7 2 3]
 [3 7 6 9 7]
 [1 3 7 3 4]
 [9 2 4 1 7]]
Salvage value:  [135 132 139 138 108]
Additional producing cost:  [212 201 227 244 232 163 205 194]
Selling prices:  [219 369 302 247 360 294 303 250]
Cost coefficient: [  -7 -168  -75   -3 -128 -131  -98  -56]
Random demand:
 [[3 5 6 7 4 7 4 4]
 [5 6 4 5 3 5 3 6]]
PMF of demand:
 [[0.1171875  0.24609375 0.20507812 0.1171875  0.20507812 0.1171875
  0.20507812 0.20507812]
 [0.24609375 0.20507812 0.20507812 0.24609375 0.1171875  0.24609375
  0.1171875  0.20507812]]


In [27]:
D = np.array([
    np.random.binomial(10,0.5,n),
    np.random.binomial(10,0.5,n)
])
D

array([[2, 6, 2, 7, 7, 5, 3, 2],
       [4, 3, 6, 7, 5, 4, 5, 3]])

### First stage
In this stage, $x$ needs to be decided before a realization of the demand $D$, which makes it a $\textit {here and now}$ decisions $\newline$
Moreover, the expected value of $Z$ should also be taken into account. To be specific, that is  
$$
Q(x) = E[Z(z)] = \sum_{i=1}^n p_i c_i z_i
$$
Therefore, we yield this problem below: 
$$min \text { } g(x,z) = b^Tx + Q(x) = b^Tx + E[Z(z)]$$
However, the demand is not known to solve for the optimal value of Q(x). Therefore, we have to decide about $\textit {not knowing}$ the demand. There are two suggestions from the specification of the assignment, guess at uncertainty and Probabilistic constraints
